# Spark Development Notebook 


add details here ... 

In [1]:
# imports 
import os 
import faker
import pandas as pd
from pyspark.sql import SparkSession
import pydeequ

In [3]:
# ensure SPARK_HOME & other environment variables exist as expected 
print(f"SPARK_HOME: {os.getenv('SPARK_HOME')}")
print(f"PATH: {os.getenv('PATH')}")
print(f"PYTHONPATH: {os.getenv('PYTHONPATH')}")
print(f"SPARK_VERSION: {os.getenv('SPARK_VERSION')}") # note, this must be set in order for pydeequ to work 

SPARK_HOME: /opt/spark-3.3.4-bin-hadoop3
PATH: /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/spark-3.3.4-bin-hadoop3/bin
PYTHONPATH: /opt/spark-3.3.4-bin-hadoop3/python:/opt/spark-3.3.4-bin-hadoop3/python/lib/py4j-3.3.4-src.zip:
SPARK_VERSION: 3.3.4


Check that the MinIO environment variables exist on the Edge Node

In [4]:
print(os.getenv('MINIO_ROOT_USER'))

minioadmin


Now, create the Spark Session against the Local Cluster running via Docker

In [5]:
# Set up the Spark session
spark = (SparkSession.builder 
    .appName("Spark Dev App") 
    .master("spark://spark-master:7077")   # Connect to the Spark cluster
    .config("spark.executor.memory", "1g")
    .config("spark.jars.packages", pydeequ.deequ_maven_coord) # used for PyDeequ
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord) # used for PyDeequ 
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")  # MinIO endpoint
    .config("spark.hadoop.fs.s3a.access.key", os.getenv('MINIO_ROOT_USER'))   # MinIO access key
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv('MINIO_ROOT_PASSWORD'))   # MinIO secret key
    .config("spark.hadoop.fs.s3a.path.style.access", "true")  # Required for MinIO
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")   # S3A connector
    .config("spark.sql.catalog.my_catalog", "org.apache.iceberg.spark.SparkCatalog")   # Iceberg catalog configuration
    .config("spark.sql.catalog.my_catalog.type", "hadoop")   # Iceberg catalog type
    #.config("spark.sql.catalog.my_catalog.warehouse", "s3a://my-bucket/iceberg")   # Iceberg warehouse path in MinIO
    .config("spark.jars", "$SPARK_HOME/jars/iceberg-spark-runtime-3.3_2.12-1.2.0.jar")   # Load Iceberg runtime JAR
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")   # Disable SSL (if MinIO is running without SSL)
    .getOrCreate()
)

# Confirm the Spark session is up and running
spark

/opt/spark-3.3.4-bin-hadoop3/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/opt/spark-3.3.4-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b229cf7c-d74d-446b-a1a2-2d5c0beb64ee;1.0
	confs: [default]
	found com.amazon.deequ#deequ;2.0.7-spark-3.3 in central
	found org.scala-lang#scala-reflect;2.12.10 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_2.12;1.1.1 in central
	fo

24/09/12 18:24:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/12 18:24:09 WARN DependencyUtils: Local jar /development/$SPARK_HOME/jars/iceberg-spark-runtime-3.3_2.12-1.2.0.jar does not exist, skipping.
24/09/12 18:24:09 INFO SparkContext: Running Spark version 3.3.4
24/09/12 18:24:09 INFO ResourceUtils: ==============================================================
24/09/12 18:24:09 INFO ResourceUtils: No custom resources configured for spark.driver.
24/09/12 18:24:09 INFO ResourceUtils: ==============================================================
24/09/12 18:24:09 INFO SparkContext: Submitted application: Spark Dev App
24/09/12 18:24:09 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: 

24/09/12 18:24:20 INFO CoarseGrainedSchedulerBackend$DriverEndpoint: Registered executor NettyRpcEndpointRef(spark-client://Executor) (172.27.0.5:49530) with ID 1,  ResourceProfileId 0
24/09/12 18:24:20 INFO CoarseGrainedSchedulerBackend$DriverEndpoint: Registered executor NettyRpcEndpointRef(spark-client://Executor) (172.27.0.4:53340) with ID 0,  ResourceProfileId 0
24/09/12 18:24:20 INFO BlockManagerMasterEndpoint: Registering block manager 172.27.0.4:37203 with 434.4 MiB RAM, BlockManagerId(0, 172.27.0.4, 37203, None)
24/09/12 18:24:20 INFO BlockManagerMasterEndpoint: Registering block manager 172.27.0.5:39663 with 434.4 MiB RAM, BlockManagerId(1, 172.27.0.5, 39663, None)
24/09/12 23:48:34 WARN HeartbeatReceiver: Removing executor 1 with no recent heartbeats: 6689315 ms exceeds timeout 120000 ms
24/09/12 23:48:34 WARN HeartbeatReceiver: Removing executor 0 with no recent heartbeats: 18452547 ms exceeds timeout 120000 ms
24/09/12 23:48:34 INFO StandaloneSchedulerBackend: Requesting t

### Spark Web UI

You can access the Spark Web UI via: http://localhost:8080 

Here, we can see that we have our application running:

![Spark UI](./SparkMaster_WebUI.png)


----------------------

#### Testing PySpark on a Local Cluster